<a href="https://colab.research.google.com/github/TC2008B-Team5/Multiagent-Systems-T5/blob/main/MESA_Team5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install -U 'mesa'

In [33]:
!pip install seaborn; solara; matplotlib; ipywidgets

                                                                                
 Usage: solara [OPTIONS] COMMAND [ARGS]...                                      
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ --help      Show this message and exit.                                      │
╰──────────────────────────────────────────────────────────────────────────────╯
╭─ Commands ───────────────────────────────────────────────────────────────────╮
│ create            Quickly create a solara script or project.                 │
│ deploy                                                                       │
│ run               Run a Solara app.                                          │
│ search            Build search index based on ssg output                     │
│ ssg               Static site generation                                     │
│ staticbuild       Experime

In [34]:
# Import Mesa modules
from mesa import Agent, Model
from mesa.agent import AgentSet
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

# Import visualization modules
from mesa.visualization import SolaraViz, make_space_component
from mesa.visualization.utils import update_counter
import solara
import numpy as np
import matplotlib.pyplot as plt


In [35]:
class CarAgent(Agent):
    """An agent representing a car moving from a parking lot to a destination parking lot."""

    def __init__(self, model):
        super().__init__(model)
        #self.pos = start_pos
        self.destination_pos = None
        self.active = True  # Indicates if the car is still moving
        self.path = []
        #self.calculate_path()
        print(f"Car {self.unique_id} created at position {self.pos}")


    def calculate_path(self):
        #Calculate a path from start_pos to destination_pos avoiding buildings and following road directions.
        # Implement a pathfinding algorithm that respects buildings and road directions
        self.path = self.find_path(self.pos, self.destination_pos)

    def find_path(self, start: tuple[int, int], goal: tuple[int, int]) -> list[tuple[int, int]]:
        #Find a valid path using A* algorithm
        open_set = {start}
        came_from = {}
        g_score = {start: 0}
        f_score = {start: self.heuristic(start, goal)}

        while open_set:
            current = min(open_set, key=lambda pos: f_score.get(pos, float('inf')))
            print(f"Current position: {current}")
            if current == goal:
                print(f"Goal reached: {goal}")
                return self.reconstruct_path(came_from, current)

            open_set.remove(current)
            for neighbor in self.get_valid_neighbors(current):
                tentative_g_score = g_score[current] + 1  # Assumes uniform cost
                if tentative_g_score < g_score.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + self.heuristic(neighbor, goal)
                    open_set.add(neighbor)
        return []

    def heuristic(self, a: tuple[int, int], b: tuple[int, int]) -> int:
        """Heuristic function for A* (Manhattan distance)."""
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def reconstruct_path(self, came_from: dict, current: tuple[int, int]) -> list[tuple[int, int]]:
        """Reconstruct the path from start to goal."""
        total_path = [current]
        while current in came_from:
            current = came_from[current]
            total_path.append(current)
        total_path.reverse()
        return total_path

    def get_valid_neighbors(self, pos: tuple[int, int]) -> list[tuple[int, int]]:
        """Get neighbors that are valid for movement."""
        neighbors = self.model.grid.get_neighborhood(pos, moore=False, include_center=False)
        valid_neighbors = []
        for neighbor in neighbors:
            if self.is_valid_move(pos, neighbor):
                valid_neighbors.append(neighbor)
        return valid_neighbors

    def is_valid_move(self, from_pos: tuple[int, int], to_pos: tuple[int, int]) -> bool:
        """Check whether moving from from_pos to to_pos is a valid move."""
        if self.model.grid.out_of_bounds(to_pos):
            print(f"Move from {from_pos} to {to_pos} is out of bounds.")
            return False

        # Get the list of agents at to_pos
        cell_contents = self.model.grid.get_cell_list_contents([to_pos])
        
        if len(cell_contents) > 0:
            print(f"Move from {from_pos} to {to_pos} is blocked by another agent.")
            return False  # Cell is occupied by another agent

        # Check for buildings
        if self.model.is_building(to_pos):
            print(f"Move from {from_pos} to {to_pos} is blocked by a building.")
            return False  # Can't move into a building

        # Allow movement into or out of parking lot
        if self.model.is_parking_lot(from_pos) or self.model.is_parking_lot(to_pos):
            print(f"Move from {from_pos} to {to_pos} is allowed as it involves a parking lot.")
            return True  # Allow movement into or out of parking lots

        # Get valid directions from the current position
        valid_directions = self.model.road_direction_layer[from_pos[0], from_pos[1]]
        if valid_directions is None:
            print(f"No valid directions from {from_pos}.")
            return False  # No valid directions from this cell
        if not isinstance(valid_directions, list):
            valid_directions = [valid_directions]  # Convert to list if necessary

        # Determine the actual movement direction
        dx = to_pos[0] - from_pos[0]
        dy = to_pos[1] - from_pos[1]
        movement_direction = None
        if dx == 1:
            movement_direction = 'E'
        elif dx == -1:
            movement_direction = 'W'
        elif dy == 1:
            movement_direction = 'N'
        elif dy == -1:
            movement_direction = 'S'

        # Check if the movement direction is allowed
        if movement_direction in valid_directions:
            print(f"Move from {from_pos} to {to_pos} in direction '{movement_direction}' is valid.")
            return True
        else:
            print(f"Move from {from_pos} to {to_pos} in direction '{movement_direction}' is invalid.")
        return False

    def step(self):
        """Advance the agent by one step."""
        print(f"Car {self.unique_id} at position {self.pos} taking a step.")
        if self.active and self.path:
            next_pos = self.path.pop(0)
            self.model.grid.move_agent(self, next_pos)
            print(f"Car {self.unique_id} moved to {next_pos}")
            if next_pos == self.destination_pos:
                self.active = False
                print(f"Car {self.unique_id} has arrived at Parking Lot at position {self.destination_pos}")
                self.remove()
        else:
            self.active = False
            print(f"Car {self.unique_id} has no path or is inactive.")
            self.remove()

    def remove(self):
        """Remove the car agent from the model and grid."""
        if self.pos is not None:
            self.model.grid.remove_agent(self)
        super().remove()

In [36]:
class TrafficLightAgent(Agent):
    """An agent representing a traffic light."""

    def __init__(self, model: Model, pos: tuple[int, int]):
        super().__init__(model)
        self.pos = pos
        self.state = 'Green'  # Initial state
        self.timer = 0
        self.durations = {'Green': 5, 'Yellow': 2, 'Red': 5}
        self.model.grid.place_agent(self, pos)

    def step(self):
        """Advance the traffic light by one step."""
        self.timer += 1
        if self.timer >= self.durations[self.state]:
            self.change_state()
            self.timer = 0

    def change_state(self):
        """Cycle through traffic light states."""
        if self.state == 'Green':
            self.state = 'Yellow'
        elif self.state == 'Yellow':
            self.state = 'Red'
        elif self.state == 'Red':
            self.state = 'Green'

In [37]:
class CityModel(Model):
    """A model representing a city grid with cars, buildings, parking lots, and traffic lights."""

    def __init__(self, num_cars, width, height, seed=None):
        super().__init__(seed=seed)

        self.num_cars = num_cars
        self.grid = MultiGrid(width, height, torus=False)
        self.steps = 0

        # Initialize property layers
        self.buildings_layer = np.full((width, height), False, dtype=bool)
        self.parking_lot_layer = np.full((width, height), False, dtype=bool)
        self.parking_lot_ids = {}
        self.road_direction_layer = np.full((width, height), None, dtype=object)
        

        # Set up buildings, parking lots, and roads
        self.setup_environment()

        # Agent sets
        self.car_agents = AgentSet([], random=self.random)
        #self.traffic_light_agents = AgentSet([], random=self.random)

        # Create traffic lights (if any)
        #self.create_traffic_lights()

        # Create cars
        self.create_cars()

    def setup_environment(self):
        """Set up buildings, parking lots, and road directions."""
        # Place buildings
        self.setup_buildings()
        # Place parking lots
        self.setup_parking_lots()
        # Set road directions
        self.setup_road_directions()

    def setup_buildings(self):
        """Place buildings on the grid."""
        # Example: Buildings occupy multiple cells
        # Define building positions
        buildings_positions = [
            # Building 1
            [
                (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 10), (2, 11),
                (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11),
                (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10),
                (5, 2), (5, 3), (5, 4), (5, 5), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11)
            ],
            # Building 2
            [
                (8, 2), (8, 3), (8, 4),
                (9, 2), (9, 3), (9, 4),
                (10, 2), (10, 3),
                (11, 2), (11, 3), (11, 4)
            ],
            # Building 3
            [
                (8, 7), (8, 9), (8, 10), (8, 11),
                (9, 7), (9, 8), (9, 9), (9, 10), (9, 11),
                (10, 7), (10, 8), (10, 9), (10, 10),
                (11, 7), (11, 8), (11, 9), (11, 10), (11, 11)
            ],
            # Building 4
            [
                (16, 2), (16, 3), (16, 4), (16, 5),
                (17, 3), (17, 4), (17, 5),
                (18, 2), (18, 3), (18, 4), (18, 5),
                (19, 2), (19, 3), (19, 4), (19, 5),
                (20, 2), (20, 3), (20, 4),
                (21, 2), (21, 3), (21, 4), (21, 5)
            ],
            # Building 5
            [
                (16, 8), (17, 8), (18, 8), (19, 8), (21, 8),
                (16, 9), (16, 10), (16, 11),
                (17, 9), (17, 10), (17, 11),
                (18, 9), (18, 10), (18, 11),
                (19, 9), (19, 10), (19, 11),
                (20, 9), (20, 10), (20, 11),
                (21, 9), (21, 10), (21, 11)
            ],
            # Building 6
            [
                (16, 16), (16, 17), (16, 18), (16, 19), (16, 20), (16, 21),
                (17, 16), (17, 18), (17, 20), (17, 21)
            ],
            # Building 7
            [
                (20, 16), (20, 17), (20, 18), (20, 20), (20, 21),
                (21, 16), (21, 17), (21, 18), (21, 19), (21, 20), (21, 21)
            ],
            # Building 8
            [
                (8, 16), (8, 17),
                (9, 16), (9, 17),
                (10, 17),
                (11, 16), (11, 17)
            ],
            # Building 9
            [
                (8, 20), (8, 21),
                (9, 20),
                (10, 20), (10, 21),
                (11, 20), (11, 21)
            ],
            # Building 10
            [
                (2, 16), (2, 17),
                (3, 16), 
                (4, 16), (4, 17),
                (5, 16), (5, 17)
            ],
            # Building 11
            [
                (2, 20), (2, 21),
                (3, 20), (3, 21),
                (4, 21),
                (5, 20), (5, 21)
            ]
        ]
        for building in buildings_positions:
            for pos in building:
                x, y = pos
                self.buildings_layer[x, y] = True  # Mark cell as building

    def setup_parking_lots(self):
        """Place parking lots on the grid."""
        parking_lot_positions = [
            (2, 9), (3, 2), (3, 17), (4, 11), (4, 20), (5, 6), (8, 8), 
            (9, 21), (10, 4), (10, 11), (10, 16), (17, 2), (17, 17), (17, 19),
            (20, 5), (20, 8), (20, 19)
        ]
        for idx, pos in enumerate(parking_lot_positions, start=1):
            x, y = pos
            self.parking_lot_layer[x, y] = True  # Mark cell as parking lot
            self.parking_lot_ids[pos] = idx

    def setup_road_directions(self):
        """Set up road directions on the grid for testing purposes."""
        width, height = self.grid.width, self.grid.height

        # Define all road positions (for testing, assume all non-building cells are roads)
        for x in range(width):
            for y in range(height):
                if not self.buildings_layer[x, y]:
                    # Allow movement in all directions
                    self.road_direction_layer[x, y] = ['N', 'S', 'E', 'W']
                    
    def assign_random_destination(self, car, exclude_pos):
        """Assign a random parking lot as the destination to the car."""
        possible_destinations = [
            pos for pos in self.parking_lot_ids.keys() if pos != exclude_pos
        ]

        if not possible_destinations:
            raise ValueError("No other parking lots available for destination.")

        destination_pos = self.random.choice(possible_destinations)
        car.destination_pos = destination_pos

        # Get the parking lot IDs
        start_lot_id = self.parking_lot_ids[exclude_pos]
        dest_lot_id = self.parking_lot_ids[destination_pos]

        print(f"Car {car.unique_id} is leaving Parking Lot {start_lot_id} at position {exclude_pos}")
        print(f"Car {car.unique_id} has destination Parking Lot {dest_lot_id} at position {destination_pos}")

    def create_cars(self):
        """Create car agents starting at parking lots."""
        # Get positions of parking lots
        available_parking_lots = list(self.parking_lot_ids.keys())

        if not available_parking_lots:
            raise ValueError("No parking lots available to assign start positions.")

        num_cars_to_create = min(self.num_cars, len(available_parking_lots))
        
        for _ in range(num_cars_to_create):
            # Select a random parking lot as the starting position
            start_pos = self.random.choice(available_parking_lots)

            # Create the car agent
            car = CarAgent(self)

            # Place the car agent at the starting parking lot
            self.grid.place_agent(car, start_pos)
            self.car_agents.add(car)
            #self.schedule.add(car)

            print(f"Car {car.unique_id} placed at position {start_pos}")

            # Remove the start position to prevent multiple cars at the same spot
            available_parking_lots.remove(start_pos)

            # Assign a random destination parking lot
            self.assign_random_destination(car, exclude_pos=start_pos)

            # Calculate the path
            car.calculate_path()

    def create_traffic_lights(self):
        """Create traffic light agents at specified positions."""
        traffic_light_positions = [
            (12, 12), (12, 0), (0, 12), (23, 12), (12, 23)
            # Add more traffic lights as needed
        ]
        for pos in traffic_light_positions:
            traffic_light = TrafficLightAgent(self, pos)
            self.traffic_light_agents.add(traffic_light)
            # Place the traffic light agent on the grid
            self.grid.place_agent(traffic_light, pos)
    
    def is_parking_lot(self, pos: tuple[int, int]) -> bool:
        """Check if a position is occupied by a parking lot."""
        x, y = pos
        return self.parking_lot_layer[x, y]

    def is_building(self, pos: tuple[int, int]) -> bool:
        """Check if a position is occupied by a building."""
        x, y = pos
        return self.buildings_layer[x, y]

    def is_valid_road_direction(self, from_pos, to_pos):
        """Check if movement from from_pos to to_pos follows the road direction."""
        from_x, from_y = from_pos
        to_x, to_y = to_pos
        direction = self.road_direction_layer[from_x, from_y]
        if direction == 'N' and to_y == from_y + 1:
            return True
        if direction == 'S' and to_y == from_y - 1:
            return True
        if direction == 'E' and to_x == from_x + 1:
            return True
        if direction == 'W' and to_x == from_x - 1:
            return True
        return False

    def step(self):
        """Advance the model by one step."""
        print(f"Model stepping. Current step: {self.steps}")
        # Activate traffic lights
        #self.traffic_light_agents.do("step")
        # Activate cars
        self.car_agents.do("step")
        # Increment step counter
        #self.steps += 1

In [38]:
@solara.component
def GridVisualization(model):
    """Custom grid visualization for the CityModel including buildings, parking lots, and agents."""
    # Ensure the visualization updates when the model changes
    update_counter.get()
    
    grid = model.grid
    width = grid.width
    height = grid.height
    
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')
    
    # Draw grid cells
    for x in range(width):
        for y in range(height):
            if model.buildings_layer[x, y]:
                # Draw buildings in blue
                rect = plt.Rectangle((x, y), 1, 1, facecolor='blue')
                ax.add_patch(rect)
            elif model.parking_lot_layer[x, y]:
                # Draw parking lots in yellow
                rect = plt.Rectangle((x, y), 1, 1, facecolor='yellow')
                ax.add_patch(rect)
            else:
                # Draw empty cells in white with black borders
                rect = plt.Rectangle((x, y), 1, 1, facecolor='white', edgecolor='black')
                ax.add_patch(rect)
    
    # Draw agents
    for agent in model.car_agents:
        x, y = agent.pos
        # Draw agents as red circles centered in the cell
        circle = plt.Circle((x + 0.5, y + 0.5), 0.3, color='red')
        ax.add_patch(circle)
    
    # Invert y-axis to match the grid orientation
    plt.gca().invert_yaxis()
    plt.close(fig)  # Close the figure to prevent duplicate displays
    solara.FigureMatplotlib(fig)

In [39]:
# Create an instance of the model
model = CityModel(num_cars=5, width=24, height=24)

# List of visualization components
components = [GridVisualization]

model_params = {
    "num_cars": 5,
    "width": 24,
    "height": 24
}
# Set up the Solara visualization
page = SolaraViz(
    model=model,
    components=components,
    name="City Model Visualization",
    model_params=model_params
)

Car 1 is leaving Parking Lot 12 at position (17, 2)
Car 1 has destination Parking Lot 13 at position (17, 17)
Current position: (17, 2)
Move from (17, 2) to (16, 2) is blocked by a building.
Move from (17, 2) to (17, 1) is allowed as it involves a parking lot.
Move from (17, 2) to (17, 3) is blocked by a building.
Move from (17, 2) to (18, 2) is blocked by a building.
Current position: (17, 1)
Move from (17, 1) to (16, 1) in direction 'W' is valid.
Move from (17, 1) to (17, 0) in direction 'S' is valid.
Move from (17, 1) to (17, 2) is blocked by another agent.
Move from (17, 1) to (18, 1) in direction 'E' is valid.
Current position: (17, 0)
Move from (17, 0) to (16, 0) in direction 'W' is valid.
Move from (17, 0) to (17, 1) in direction 'N' is valid.
Move from (17, 0) to (18, 0) in direction 'E' is valid.
Current position: (18, 1)
Move from (18, 1) to (17, 1) in direction 'W' is valid.
Move from (18, 1) to (18, 0) in direction 'S' is valid.
Move from (18, 1) to (18, 2) is blocked by a 

In [40]:
# Display the page
page

Cannot show ipywidgets in text